In [27]:
#imports
from scipy import ndimage 
import imageio
#from scipy import misc
import numpy as np
import pandas as pd
import os


In [28]:
#loop through face labels and black out all that are marked as a bystander
def obfuscate_faces(faces_df,frame_id,img_path,labels_df,obfuscated_path):
    #load img from source (using frame_id)
    
    if "day" in img_path:
        img_file_path = img_path + str(frame_id).zfill(3) + '.jpg'
    else:
        img_file_path = img_path + str(frame_id) + '.jpg'
    
    try:
        frame_img = imageio.imread(img_file_path)
    except:
        print(img_file_path)
        #return faces_df
        return []
    #print(frame_img.shape)
    img_w, img_h = frame_img.shape[1],frame_img.shape[0]
    
    
    face_ids = np.unique(faces_df['face'].values)
    
    #loop over faces
    for face_id in face_ids:
        curr_face_idx = faces_df['face'].values == face_id    
        curr_face_df  = faces_df[curr_face_idx]
        
        curr_face_label_idx = labels_df['face'].values == face_id    
        curr_face_label_df  = labels_df[curr_face_label_idx]
        
        #if face didn't pass confidence test before, won't have a label, so skip
        if curr_face_label_df.empty:
            continue
        
        #only obscure bystanders 
        if curr_face_label_df['Bystander Label'].values[0] == 0:
            continue
        
        #use landmarks to get 2d bounding box of face in image frame
        bb_left   = int(curr_face_df[' x_0'])
        bb_right  = int(curr_face_df[' x_16'])
        bb_top    = int(curr_face_df[' y_24'])
        bb_bottom = int(curr_face_df[' y_8'])

        #print(bb_top)
        #print(bb_bottom)
        if bb_left < 0 | bb_right >= img_w | bb_top < 0 | bb_bottom >= img_h:
            print('Out of bounds: Frame '+str(frame_id) + ' Face ' + str(face_id))
            continue
        
        #black out the face region
        frame_img[bb_top:bb_bottom,bb_left:bb_right,:] = 0
        
    #save the image with obfuscated faces
    try:
        imageio.imwrite(obfuscated_path + str(frame_id)+'.jpg',frame_img)
    except:
        #print(bb_left)
        #print(bb_right)
        #print(bb_top)
        #print(bb_bottom)
        print('Error writing cropped face')                                                  
                                                        
    return faces_df

In [29]:
#read in file

dataset = 'day1_test1'
#dataset = 'day1_test2'
#dataset = 'day1_test3'

obfuscated_dataset = './' + dataset + '_obfuscated' + '/'
if not os.path.exists(obfuscated_dataset):
    os.makedirs(obfuscated_dataset)

feature_path = './processed_' + dataset + '/openface_features.csv'
img_path     = './' + dataset + '/'
labels_path  = './' + dataset + '_predictions.csv'

face_features_df = pd.read_csv(feature_path)
labels_df        = pd.read_csv(labels_path)


In [30]:
frame_ids = np.unique(face_features_df['frame'].values)


#loop over frames
for frame_id in frame_ids:
    curr_frame_face_idx = face_features_df['frame'].values == frame_id                            
    curr_df = face_features_df[curr_frame_face_idx]
    
    #should also get curr labels for this frame
    curr_labels_face_idx = labels_df['frame'].values == frame_id                            
    curr_labels_df       = labels_df[curr_labels_face_idx]
    
    #call write_out_obfuscate
    obfuscate_faces(curr_df,frame_id,img_path,curr_labels_df,obfuscated_dataset)

C:\Users\Brendan David-John\AppData\Local\Temp\ipykernel_12432\4058854017.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frame_img = imageio.imread(img_file_path)
